In [ ]:
import pandas as pd
import random
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.cluster import DBSCAN
import math
import statistics
pd.options.mode.chained_assignment = None  # default='warn'
import itertools

#Convert CSV to Dataframe
data = pd.read_csv("C:/Users/Siloux/Downloads/pairwise_user_experiment (5).csv", header=[0])
data["FOLDER"] = data["IN FILE"].apply(lambda x: "")
data["LAST MOD BY"] = data["LAST MOD BY"].apply(lambda x: x.split(" ")[0])
data

#### Import Confidence Functions and Create List of Functions

In [2]:
from confidence_functions.confidence import *
 
confidence_funcs = [get_pairwise_iou, get_silhoutte_confidence, get_silhoutte_users_confidence, majority_vote]
confidence_funcs

[<function confidence_functions.confidence.get_pairwise_iou(df, users)>,
 <function confidence_functions.confidence.get_silhoutte_confidence(df, users)>,
 <function confidence_functions.confidence.get_silhoutte_users_confidence(df, users)>,
 <function confidence_functions.confidence.majority_vote(df, users, chunk_length=1)>]

### permuatation code

In [3]:
def permutate_users(df, num_users, clip_name, confidence_func):
    #Get data for indivual clip
    results = []
    df = df[df["IN FILE"] == clip_name]
    users = df["LAST MOD BY"].unique()
    
    #For each insertion order, get the confidence via some confidence function
    for user_permutation in itertools.permutations(users, num_users):
        results.append(confidence_func(df, user_permutation))
        
    results = np.mean(results)
    return results
    
#permutate_users(data, 2, "XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3", get_silhoutte_users_confidence)

### Clip Spefific Experiment

In [4]:
def determine_confidence_by_clip(df, clip_name, confidence_funcs):
    results = {}
    df = df[df["IN FILE"] == clip_name]
    users = df["LAST MOD BY"].unique()
    for confidence_func in confidence_funcs:
        print("working with function: " + confidence_func.__name__)
        results[confidence_func] = {}
        
        for user_count in range(len(users)):
            results[confidence_func][user_count+1]= permutate_users(df, user_count+1, clip_name, confidence_func)
    return results

def determine_confidence_by_func(df, confidence_funcs):
    results = {}
    users = df["LAST MOD BY"].unique()
    files = df["IN FILE"].unique()
    for func in confidence_funcs:
        results[func] = {}
        for user_count in range(len(users)):
            confidence = np.array([])
            for file in files:
                if (user_count < len(df[df["IN FILE"] == file]["LAST MOD BY"].unique())):
                       confidence = np.append(confidence, permutate_users(df, user_count+1, file, func))
            print(confidence)
            confidence = confidence.mean()
            results[func][user_count+1] = confidence
    return results
        
#determine_confidence_by_clip(data, "XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3", confidence_funcs)
#determine_confidence_by_func(data, confidence_funcs)

# Full Experiment

In [ ]:
def run_experiment(df, confidence_funcs):
    #Get the adverage confidence over number of users for each file
    results = {"by_clip": {}, "Over_all_clips": {}}
    filenames = df["IN FILE"].unique()
    for file in filenames:
        print("working with file: " + file)
        results["by_clip"][file] = determine_confidence_by_clip(df, file, confidence_funcs)
    
    print("Get adverage over all files")
    results["Over_all_clips"] = determine_confidence_by_func(df, confidence_funcs)
        
        
    
    return results

experiment_results = run_experiment(data, confidence_funcs)
experiment_results

working with file: XC425555_-_Screaming_Piha_-_Lipaugus_vociferans.mp3
working with function: get_pairwise_iou


C:\Users\seanh\Desktop\e4e\passive-acoustic-biodiversity\Pyrenote_clustering\confidence_functions\confidence.py:32: RuntimeWarning: Mean of empty slice.
  iou_scores = iou_scores.mean()
C:\ProgramData\Miniconda3\envs\species-id\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
def generate_graph(experiment_results):
    for filename in experiment_results:
        #For each filename, plot the performance of each confidence function
        for confidence_func in experiment_results[filename]:
            confidence_dirc = experiment_results[filename][confidence_func]
            users_count = list(confidence_dirc.keys())
            score = list(confidence_dirc.values())
            plt.plot(users_count, score, label=(confidence_func.__name__))
        
        #Throw in some fancy formatting
        plt.legend(loc='best')
        plt.title(filename)
        plt.xlabel("User Count")
        plt.ylabel("Average Confidence of Function")
        plt.show()

generate_graph(experiment_results["by_clip"])

In [ ]:
def generate_graph_all(experiment_results):
    #For each filename, plot the performance of each confidence function
    for confidence_func in experiment_results:
        confidence_dirc = experiment_results[confidence_func]
        users_count = list(confidence_dirc.keys())
        score = list(confidence_dirc.values())
        plt.plot(users_count, score, label=(confidence_func.__name__))

    #Throw in some fancy formatting
    plt.legend(loc='best')
    #plt.title(filename)
    plt.xlabel("User Count")
    plt.ylabel("Average Confidence of Function")
    plt.show()
    
    
    
    
generate_graph_all(experiment_results["Over_all_clips"])